In [1]:
using DrWatson
quickactivate(@__DIR__)

using NetHeider
using LinearAlgebra
using LightGraphs

In [2]:
function single_update(params::Params, attr::Matrix{Int}, signs::Matrix{Float64}, triads)
    if length(triads) == 0
        return attr, signs, triads
    end
    #choose triad
    triad = rand(triads)
    triad_links = (signs[triad[1],triad[2]], signs[triad[2],triad[3]], signs[triad[3],triad[1]])
    triad_links_inds = ((triad[1],triad[2]), (triad[2],triad[3]), (triad[3],triad[1]))

    #determine triad type
    is_balanced = prod(triad_links) > 0

    if !is_balanced
        s = sum(triad_links)
        if s == 1 # triad with 1 neg link
            if rand() < params.pn #change neg link
                neg_link_ind = findfirst(triad_links .== -1)
                change_link = triad_links_inds[neg_link_ind]
            else
                pos_link_inds = findall(triad_links .== 1)
                change_link = triad_links_inds[rand(pos_link_inds)]
            end
        else # triad with 3 neg links
            change_link = rand(triad_links_inds)
        end

        pr = signs[change_link...] > 0 ? params.pr_pos : params.pr_neg
        if rand() < pr
            # remove connection
            rem_edge!(net, change_link...)

            triads = get_undir_triads(net)
        else #change attribute
            #choose agent
            ind_1 = rand(1:2)
            agent_1 = change_link[ind_1]
            agent_2 = ind_1 == 1 ? change_link[2] : change_link[1]

            #find set of attributes
            v = get_degeneracy(params.attr)
            dif = attr[agent_1,:] .- attr[agent_2,:]
            if signs[change_link...] > 0 #this link is positive. We want to make it negative
                #find set of not completely different attributes
                attr_inds = findall(abs.(dif) .!= v-1)

                #exclude attributes that cannot become more different (for instance attribute 1 cannot become smaller)
                attr_inds = [attr_ind for (i, attr_ind) in enumerate(attr_inds) if dif[i] == 0 || (attr[agent_1, attr_ind] != 1 && attr[agent_1, attr_ind] != v)]
            else #this link is negative. We want to make it positive
                #find set of not the same attributes
                attr_inds = findall(attr[agent_1,:] .!= attr[agent_2,:])
            end

            attr_ind = rand(attr_inds)
            if signs[change_link...] > 0 #this link is positive. We want to make it negative
                    
                if dif[attr_ind] == 0
                    if attr[agent_1, attr_ind] == 1
                        possible_new_vals = [attr[agent_1, attr_ind] + 1]
                    elseif attr[agent_1, attr_ind] == v
                        possible_new_vals = [attr[agent_1, attr_ind] - 1]
                    else possible_new_vals = [attr[agent_1, attr_ind] - 1, attr[agent_1, attr_ind] + 1]
                    end
                else
                    possible_new_vals = [attr[agent_1, attr_ind] + sign(dif[attr_ind])]
                end
            else #this link is negative. We want to make it positive
                possible_new_vals = [attr[agent_1, attr_ind] - sign(dif[attr_ind])]
            end

            attr[agent_1, attr_ind] = rand(possible_new_vals)
        end
    end
    return attr, signs, triads
end

single_update (generic function with 1 method)

In [5]:
attribute = OrderedAttributes(3, 0.25, 3)
attr = [1 1 1;
    1 1 3; 
    1 1 2]
signs = sign.(Symmetric(get_attribute_layer_weights(attribute, attr)))
net = NetHeider.generate_complete_network(3)
triads = get_undir_triads(net)

params = Params()
params.attr = attribute
params

Params(OrderedAttributes(3, 0.25, 3), 3, "complete", 0, 0.5, 0.5, 0.5, 0.5, false, 1000, 10, 100, Dates.Date("2022-04-21"), "results.csv", 60, 0.25, 60)

In [6]:
signs

3×3 Matrix{Float64}:
  0.0  -1.0  1.0
 -1.0   0.0  1.0
  1.0   1.0  0.0

In [68]:
#neg link will be chosen for certain
params.pn = 1.


1.0

In [71]:
#connection will be surely removed
params.pr_neg = 1.
attr, signs, triads = single_update(params, attr, signs, triads)
length(triads) == 0

true